In [3]:
import pandas as pd

FILE = "D:/Ibo/SFP/SFP_Notebook/car_data_csv.csv"

def replace_by_header(column_name, filename, value, isRegex):      
    df = pd.read_csv(filename, sep=';')
    if column_name in df.columns:
        df[column_name] = df[column_name].replace(value, '', regex=isRegex)
    df.to_csv(filename, index = False, sep=';')
    
replace_by_header("Schadstoffklasse", FILE, "Euro", False)
replace_by_header("CO₂-Emissionen2",FILE, "g/(komb.)", False)
replace_by_header("Leistung",FILE, r"\(\d+PS\)", True)

In [4]:
import pandas as pd
from datetime import datetime
import re

FILE = "D:/Ibo/SFP/SFP_Notebook/car_data_csv.csv"

def clean_column(column_name, filename):
    df = pd.read_csv(filename, sep=';')
    if column_name in df.columns:
        df[column_name] = df[column_name].apply(lambda x: 24 if x == 'Neu' else (berechne_differenz(x, True) if re.match(r"\d{2}/\d{4}", x) else berechne_differenz(x, False)))
    df.to_csv(filename, index = False, sep=';')

def berechne_differenz(datum, monate):
    datum = pd.to_datetime(datum)
    aktuelles_datum = datetime.now()  
    if monate:
        differenz = (datum.year - aktuelles_datum.year) * 12 + datum.month - aktuelles_datum.month
    else:
        differenz = aktuelles_datum.year - datum.year
    return differenz

clean_column("Erstzulassung", FILE)
clean_column("HU", FILE)

In [5]:
FILE = "D:/Ibo/SFP/SFP_Notebook/car_data_csv.csv"

def replace_by_header_contains(column_name, filename, substring, new_value):
    df = pd.read_csv(filename, sep=';')
    if column_name in df.columns:
        df[column_name] = df[column_name].apply(lambda x: new_value if substring in x else x)
    df.to_csv(filename, index=False, sep=';')

replace_by_header_contains("Kraftstoff", FILE, "Benzin", "Benzin")
replace_by_header_contains("Kraftstoff", FILE, "Super", "Benzin")
replace_by_header_contains("Kraftstoff", FILE, "Diesel", "Diesel")
replace_by_header_contains("Kraftstoff", FILE, "gas", "Gas")